# 🦌 ELK App 🚀

This is a high-level widget for interacting with ELK Transformers and ELK Diagrams

In [ ]:
import json
import pathlib

import ipywidgets
import networkx
import traitlets

import ipyelk
import ipyelk.nx
import ipyelk.tools

In [ ]:
hier_tree = networkx.readwrite.json_graph.node_link_graph(
    json.loads(pathlib.Path("hier_tree.json").read_text(encoding="utf-8"))
)
hier_ports = networkx.readwrite.json_graph.node_link_graph(
    json.loads(pathlib.Path("hier_ports.json").read_text(encoding="utf-8"))
)

hier_xelk = ipyelk.nx.XELK(source=(hier_ports, hier_tree))
hier_elk = ipyelk.ElkDiagram(hier_xelk.to_dict())

In [ ]:
class ToggleCollapsedBtn(ipyelk.tools.ToolButton):
    @traitlets.default("description")
    def _default_description(self):
        return "Toggle Collapsed"

    def handler(self, *args):
        diagram = self.app.diagram
        transformer = self.app.transformer
        graph, tree = transformer.source

        for element_id in diagram.selected:
            if element_id in tree:
                for child in tree.neighbors(element_id):
                    state = tree.nodes[child].get("hidden", False)
                    tree.nodes[child]["hidden"] = not state
                diagram.value = transformer.to_dict()

In [ ]:
elk_app = ipyelk.Elk(transformer=hier_xelk, layout=dict(display="flex", flex="1"))
toggle = ipyelk.tools.ToggleCollapsedBtn(app=elk_app)
box = ipywidgets.VBox(
    [
        ipywidgets.HBox(
            [ipywidgets.HTML("<h2>👇 click a group node then click 👉</h2>"), toggle]
        ),
        elk_app,
    ],
    layout=dict(height="100%"),
)
box

## Custom Styling For Collaping Hierarchical Example

The style dictionary takes the first key and adds a parent selector based on the current
`StyleWidget`'s id. This helps to namespace the css selectors but also means that the
leading space in the style dictionary keys is intentional and is a descendant selector
from the root.

The `.slack-port` and `.slack-edge` are custom css classes applied by the XELK
transformer when collapsing a node causes either a source or destination of a tunneling
edges to disapear requiring the introduction of a new edge and port at a higher level.

In [ ]:
elk_app.style = {
    " rect": {"opacity": ".75", "transition": "all 0.2s"},
    " .slack-port>rect": {"fill": "red", "opacity": ".25", "transition": "all 0.2s"},
    " .slack-edge>path": {
        "stroke": "red",
        "opacity": ".25",
        "stroke-dasharray": "4",
        "transition": "all 0.2s",
        "stroke-width": "2px",
    },
    " .slack-edge>path.edge.arrow": {"fill": "red", "opacity": ".25"},
}

## 🦌 Learn More 📖

- [🦌 Introducing ELK 👋](./00_Introduction.ipynb)
- [🦌 Linking ELK Diagrams 🔗](./01_Linking.ipynb)
- [🦌 ELK Transformer 🤖](./02_Transformer.ipynb)
- [🦌 ELK App 🚀](./03_App.ipynb)
- [🦌 Interactive ELK App 🕹️](./04_Interactive.ipynb)